In [19]:
import pandas as pd
import numpy as np
from collections import Counter 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import copy
from tqdm import tqdm
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import scipy.stats as stats
from sklearn.feature_selection import f_regression


COLUMN = 'Mesorhizobium sp. 113-1-2' #sys.argv[1] # Bacteria or virus name.

print(COLUMN)
BLOOD_MICROBIOME_DIR = '/home/groups/dpwall/briannac/blood_microbiome/'
MICROBE_FILE = BLOOD_MICROBIOME_DIR + 'data/kraken_align/microbe_filtered_species.tsv'

BAM_MAPPINGS_FILE = BLOOD_MICROBIOME_DIR + 'data/bam_mappings.csv'
COEFF_OUT_DIR = BLOOD_MICROBIOME_DIR + ('intermediate_files/f_regression/%s.txt' % (COLUMN.replace(' ', '_').replace('/', '_')))

bam_mappings = pd.read_csv(BAM_MAPPINGS_FILE, sep='\t', index_col=1)
bam_mappings['child'] = 1.0*(bam_mappings.relationship=='sibling')
bam_mappings['autism'] = 1.0*(bam_mappings.derived_affected_status=='autism')

df_microbe = pd.read_table(MICROBE_FILE, index_col=0).transpose()



bam_mappings['sample_id'] = bam_mappings.index
exog = pd.get_dummies(bam_mappings.loc[df_microbe.index][['bio_seq_source', 'sex_numeric', 'child', 'autism', 
                                                          'sequencing_plate', 'family', 'sample_id']], drop_first=False, dummy_na=True).astype(float)
exog = exog.drop(['bio_seq_source_LCL', 'sex_numeric_1.0'], axis=1)
exog = exog[exog.columns[exog.sum()!=0]]
exog_scale = pd.DataFrame(StandardScaler().fit(exog).transform(exog))
exog_scale.columns = exog.columns
exog_scale.index = exog.index
exog_scale.columns = exog.columns #[c.replace('.', '_').replace('-', '_') for c in exog_scale.columns]
exog = exog_scale


print(COLUMN)
endog = np.log10(df_microbe[[COLUMN]]+1)
endog = StandardScaler().fit(endog).transform(endog)
min_pval = 0
sig_cols = []
new_exog = exog
new_endog = endog
_, pvals = f_regression(new_exog, new_endog)
cols_left = exog.columns
ols_model = sm.OLS(endog, exog[[cols_left[np.argmin(pvals)]]]).fit()
new_endog = ols_model.resid
min_pval = ols_model.f_pvalue


# Perform F-regression.
while min_pval < (.05/len(df_microbe.columns)):
    sig_cols = sig_cols + [cols_left[np.argmin(pvals)]]
    print(sig_cols[-1])
    old_model = copy.deepcopy(ols_model)
    cols_left = [c for c in exog.columns if c not in sig_cols]
    _, pvals = f_regression(exog[cols_left], new_endog)
    new_exog = exog[sig_cols + [cols_left[np.argmin(pvals)]]]
    try: ols_model = sm.OLS(endog, new_exog).fit()
    except: ols_model = sm.OLS(endog, new_exog[new_exog.columns[::-1]]).fit()
    new_endog = ols_model.resid
    min_pval = anova_lm(old_model, ols_model)['Pr(>F)'][1]
    print(min_pval)
results = [(COLUMN, i,j) for i,j in zip(sig_cols, ols_model.params) if 'sample_id' not in i]
pd.DataFrame(results).to_csv(COEFF_OUT_DIR, header=None)

Mesorhizobium sp. 113-1-2
Mesorhizobium sp. 113-1-2
sequencing_plate_MTR-00011


/share/software/user/open/py-scikit-learn/0.19.1_py36/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/users/briannac/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


2.3854492907528377e-68
sequencing_plate_MTR-00017
3.6519906525404843e-68
sequencing_plate_MTR-00018
6.705521926821145e-76
sequencing_plate_MTR-00009
4.226281077411177e-82
sequencing_plate_MTR-00016
1.0171279353523027e-85
sequencing_plate_MTR-00014
1.2444298042080857e-79
sequencing_plate_MTR-00012
2.443875876981966e-82
sequencing_plate_MTR-00013
2.1696333967279575e-93
sequencing_plate_MTR-00015
5.589631156072598e-88
sequencing_plate_MTR-00010
6.850617966553152e-29
sample_id_05C43231-2
5.819897217156865e-29
sample_id_05C43229-2
4.8677883168543137e-29
sample_id_07C62325-2
7.120130400169855e-22
sequencing_plate_MTR-00025
2.8093074734440422e-21
family_AU1724
6.344917822351206e-07
family_AU2140
1.604743074390374e-07
family_AU3348
9.755922386102303e-07
sample_id_09C85280
5.107451592187714e-08
sample_id_06C59433
1.2132608129808057e-06
sample_id_06C61538
7.683444085039259e-07
family_AU2793
4.4152045544261766e-07
family_AU2904
7.5393577606272e-08
family_AU3052
2.6716224927747383e-08
family_AU376

In [20]:
print(COLUMN)
endog = np.log10(df_microbe[[COLUMN]]+1)
endog = StandardScaler().fit(endog).transform(endog)
min_pval = 0
sig_cols = []
new_exog = exog
new_endog = endog
_, pvals = f_regression(new_exog, new_endog)
cols_left = exog.columns
ols_model = sm.OLS(endog, exog[[cols_left[np.argmin(pvals)]]]).fit()
new_endog = ols_model.resid
min_pval = ols_model.f_pvalue


# Perform F-regression.
while min_pval < (.05/len(df_microbe.columns)):
    sig_cols = sig_cols + [cols_left[np.argmin(pvals)]]
    print(sig_cols[-1])
    old_model = copy.deepcopy(ols_model)
    cols_left = [c for c in exog.columns if c not in sig_cols]
    _, pvals = f_regression(exog[cols_left], new_endog)
    new_exog = exog[sig_cols + [cols_left[np.argmin(pvals)]]]
    try: ols_model = sm.OLS(endog, new_exog).fit()
    except: ols_model = sm.OLS(endog, new_exog[new_exog.columns[::-1]]).fit()
    new_endog = ols_model.resid
    min_pval = anova_lm(old_model, ols_model)['Pr(>F)'][1]
    print(min_pval)
results = [(COLUMN, i,j) for i,j in zip(sig_cols, ols_model.params) if 'sample_id' not in i]
pd.DataFrame(results).to_csv(COEFF_OUT_DIR, header=None)

Mesorhizobium sp. 113-1-2
sequencing_plate_MTR-00011
2.3854492907528377e-68
sequencing_plate_MTR-00017
3.6519906525404843e-68
sequencing_plate_MTR-00018
6.705521926821145e-76
sequencing_plate_MTR-00009
4.226281077411177e-82
sequencing_plate_MTR-00016
1.0171279353523027e-85
sequencing_plate_MTR-00014
1.2444298042080857e-79
sequencing_plate_MTR-00012
2.443875876981966e-82
sequencing_plate_MTR-00013
2.1696333967279575e-93
sequencing_plate_MTR-00015
5.589631156072598e-88
sequencing_plate_MTR-00010
6.850617966553152e-29
sample_id_05C43231-2
5.819897217156865e-29
sample_id_05C43229-2
4.8677883168543137e-29
sample_id_07C62325-2
7.120130400169855e-22
sequencing_plate_MTR-00025
2.8093074734440422e-21
family_AU1724
6.344917822351206e-07
family_AU2140
1.604743074390374e-07
family_AU3348
9.755922386102303e-07
sample_id_09C85280
5.107451592187714e-08
sample_id_06C59433
1.2132608129808057e-06
sample_id_06C61538
7.683444085039259e-07
family_AU2793
4.4152045544261766e-07
family_AU2904
7.5393577606272e